# Object Detection with Cofar-10 Dataset Using ResNet50 Model

## dependencies

In [6]:
from zipfile import ZipFile
import tensorflow as tf

## dataset

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle competitions download -c cifar-10

 93% 668M/715M [00:06<00:00, 61.1MB/s]
100% 715M/715M [00:06<00:00, 109MB/s] 


In [3]:
!ls


cifar-10.zip  kaggle.json  sample_data


In [8]:
# unzipping the dataset
dataset = "/content/cifar-10.zip"

with ZipFile(dataset, "r") as zip:
  zip.extractall()
  print("succesfully extracted the dataset")

succesfully extracted the dataset
